In [10]:
import numpy as np
import torch
import torch.nn as nn


class NEAT:
    def __init__(self, population_size, input_size, output_size):
        self.population_size = population_size
        self.input_size = input_size
        self.output_size = output_size
        self.population = []
        self.species = []
        self.generation = 1

        self.mutate_weights_rate = 0.8
        self.add_node_rate = 0.03
        self.add_connection_rate = 0.05
        self.species_threshold = 3.0
        self.compatibility_threshold = 3.0
        self.max_stagnation = 15

        # Initialize the population
        for _ in range(population_size):
            genome = Genome(input_size, output_size)
            self.population.append(genome)

    def crossover(self, parent1, parent2):
        child = Genome(self.input_size, self.output_size)
        if parent2.fitness > parent1.fitness:
            parent1, parent2 = parent2, parent1

        for connection_id, gene1 in parent1.connections.items():
            gene2 = parent2.connections.get(connection_id)
            if gene2 is None or np.random.uniform() < 0.5:
                child.add_connection(gene1.copy())
            else:
                child.add_connection(gene2.copy())

        for node_id, node in parent1.nodes.items():
            child.add_node(node.copy())

        return child


    def mutate_weights(self, genome):
        for gene in genome.connections.values():
            if np.random.uniform() < self.mutate_weights_rate:
                gene.weight += np.random.uniform(-1, 1)

    def mutate_add_node(self, genome):
        if np.random.uniform() < self.add_node_rate:
            connection = np.random.choice(list(genome.connections.values()))
            genome.add_node(connection)

    def mutate_add_connection(self, genome):
        if np.random.uniform() < self.add_connection_rate:
            genome.add_connection()

    def speciate(self):
        for species in self.species:
            species.genomes = []

        for genome in self.population:
            found = False
            for species in self.species:
                if genome.distance(species.representative) < self.compatibility_threshold:
                    species.genomes.append(genome)
                    found = True
                    break

            if not found:
                new_species = Species(genome)
                self.species.append(new_species)

    def reproduce(self):
        self.speciate()
        new_population = []

        for species in self.species:
            species.genomes.sort(key=lambda x: x.fitness, reverse=True)

            if species.best_genome.fitness > species.top_fitness:
                species.top_fitness = species.best_genome.fitness
                species.stagnation = 0
            else:
                species.stagnation += 1

            if species.stagnation < self.max_stagnation:
                try:
                    num_offspring = int(np.floor(species.average_fitness / self.total_average_fitness * self.population_size))
                except:
                    num_offspring = 0
                if num_offspring < 1:
                    num_offspring = 1

                for _ in range(num_offspring):
                    if np.random.uniform() < 0.25:
                        child = species.best_genome.copy()
                    else:
                        parent1 = np.random.choice(species.genomes)
                        if np.random.uniform() < 0.9:
                            parent2 = np.random.choice(species.genomes)
                            child = self.crossover(parent1, parent2)
                        else:
                            child = parent1.copy()

                    self.mutate_weights(child)
                    self.mutate_add_node(child)
                    self.mutate_add_connection(child)

                    new_population.append(child)

        self.population = new_population
        self.generation += 1

    def run(self, num_generations):
        for _ in range(num_generations):
            self.evaluate_fitness()
            self.reproduce()

    def evaluate_fitness(self):
        for genome in self.population:
            genome.fitness = np.random.uniform()

        self.species.sort(key=lambda x: x.top_fitness, reverse=True)
        self.total_average_fitness = 0.0
        for species in self.species:
            species.calculate_average_fitness()
            self.total_average_fitness += species.average_fitness

class Genome:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.connections = {}
        self.nodes = {}
        self.fitness = 0.0

        # Create input and output nodes
        for i in range(input_size):
            self.nodes[i] = Node(i)

        for i in range(input_size, input_size + output_size):
            self.nodes[i] = Node(i)

        # Connect input nodes to output nodes
        for i in range(input_size):
            for j in range(input_size, input_size + output_size):
                self.add_connection(Connection(i, j))

    def add_node(self, connection):
        new_node_id = len(self.nodes)
        new_node = Node(new_node_id)

        self.nodes[new_node_id] = new_node

        connection.enabled = False
        input_node_id = connection.input_node.node_id
        output_node_id = connection.output_node.node_id
        self.add_connection(Connection(input_node_id, new_node_id, weight=1.0))
        self.add_connection(Connection(new_node_id, output_node_id, weight=connection.weight))



    def add_connection(self, connection):
        input_id = connection.input_node_id
        output_id = connection.output_node_id

        if output_id in self.nodes:
            for existing_connection in self.connections.values():
                if existing_connection.input_node_id == input_id and existing_connection.output_node_id == output_id:
                    return

        connection_id = (input_id, output_id)
        self.connections[connection_id] = connection

    def distance(self, other):
        matching_genes = 0
        disjoint_genes = 0
        weight_diff = 0.0

        connection_ids1 = set(self.connections.keys())
        connection_ids2 = set(other.connections.keys())
        disjoint_genes += len(connection_ids1.symmetric_difference(connection_ids2))

        for connection_id in connection_ids1.intersection(connection_ids2):
            gene1 = self.connections[connection_id]
            gene2 = other.connections[connection_id]
            matching_genes += 1
            weight_diff += abs(gene1.weight - gene2.weight)

        if matching_genes > 0:
            weight_diff /= matching_genes

        max_genome_size = max(len(connection_ids1), len(connection_ids2))
        excess_genes = max(len(connection_ids1) - matching_genes, len(connection_ids2) - matching_genes)

        return (disjoint_genes / max_genome_size) + (excess_genes / max_genome_size) + (weight_diff / 2.0)

    def copy(self):
        copy_genome = Genome(self.input_size, self.output_size)
        copy_genome.connections = {k: v.copy() for k, v in self.connections.items()}
        copy_genome.nodes = {k: v.copy() for k, v in self.nodes.items()}
        copy_genome.fitness = self.fitness
        return copy_genome


class Node:
    def __init__(self, node_id, input_node=None):
        self.node_id = node_id
        self.input_node = input_node

    def copy(self):
        return Node(self.node_id, self.input_node)




class Connection:
    def __init__(self, input_node_id, output_node_id, weight=None, enabled=True):
        self.input_node_id = input_node_id
        self.output_node_id = output_node_id
        self.weight = weight if weight is not None else np.random.uniform(-1, 1)
        self.enabled = enabled

    def copy(self):
        return Connection(self.input_node_id, self.output_node_id, self.weight, self.enabled)


class Species:
    def __init__(self, representative):
        self.representative = representative
        self.genomes = []
        self.top_fitness = 0.0
        self.stagnation = 0

    @property
    def best_genome(self):
        return max(self.genomes, key=lambda x: x.fitness)

    @property
    def calculate_average_fitness(self):
        return sum(genome.fitness for genome in self.genomes) / len(self.genomes)



# Example usage
population_size = 100
input_size = 10
output_size = 5
num_generations = 50

neat = NEAT(population_size, input_size, output_size)
neat.run(num_generations)


AttributeError: 'NoneType' object has no attribute 'node_id'